In [1]:
import numpy as np
import copy

# Week 1: Introduction to Evolutionary Tree Construction

In [2]:
'''
Distances Between Leaves Problem: Compute the distances between leaves in a weighted tree.
Input:  An integer n followed by the adjacency list of a weighted tree with n leaves.
Output: An n x n matrix (di,j), where di,j is the length of the path between leaves i and j.
'''

def Distances_Matrix(n, adjacency_list):
    if type(adjacency_list) == str:
        adjacency_list = adjacency_list.split('\n')
        
    graph        = dict()
    graph_weight = dict()
    for adjacency in adjacency_list:
        adjacency = adjacency.split(':')
        graph_weight[adjacency[0]] = int(adjacency[1])
        adjacency = adjacency[0].split('->')
        adjacency[0], adjacency[1] = int(adjacency[0]), int(adjacency[1])
        if adjacency[0] in graph:
            graph[adjacency[0]].append(adjacency[1])
        else:
            graph[adjacency[0]] = [(adjacency[1])]
    
    length_matrix = np.full([n,n], 0, int)

    for from_ in range(n):
        weight_row = [0] * (max(graph.keys()) + 1)
        froms      = [from_]
        
        while len(froms) != 0:
            
            next_froms = []
            for node in froms:
                tos = graph[node]
                
                for to in tos:
                    if to != from_:
                        if (to not in range(n)) & (weight_row[to] == 0):
                            next_froms.append(to)
                        
                        graph_weight_key = str(node) + '->' + str(to)
                        weight           = graph_weight[graph_weight_key]
                        weight_row[to]   = weight_row[node] + weight
                        
            froms = next_froms
        length_matrix[from_, :] = weight_row[: n]

    return(length_matrix)

# Test
n = 4
adjacency_list = '''0->4:11
1->4:2
2->5:6
3->5:7
4->0:11
4->1:2
4->5:4
5->4:4
5->3:7
5->2:6'''

Distances_Matrix(n, adjacency_list)

array([[ 0, 13, 21, 22],
       [13,  0, 12, 13],
       [21, 12,  0, 13],
       [22, 13, 13,  0]])

In [3]:
'''
Code Challenge: Solve the Limb Length Problem.
Input: An integer n, followed by an integer j between 0 and n - 1, followed by a space-separated additive distance matrix D (whose elements are integers).
Output: The limb length of the leaf in Tree(D) corresponding to row j of this distance matrix (use 0-based indexing).
'''

def Limb_Length(j, length_matrix):
    if type(length_matrix) == str:
        length_matrix = length_matrix.replace('\n', ' ')
        length_matrix = length_matrix.split(' ')
        length_matrix = list(map(int, length_matrix))
        length_matrix = np.array(length_matrix).reshape(n, n)

    min_length = float('Inf')
    for i in range(length_matrix.shape[0]):
        if i != j:
            for k in range(length_matrix.shape[0]):
                if (k != j) & (k != i):
                    length = (length_matrix[i, j] + length_matrix[j, k] - length_matrix[i, k]) / 2
                    if length < min_length:
                        min_length = int(length)
    
    return(min_length)

# Test
n = 4
j = 1
length_matrix = '''0 13 21 22
13 0 12 13
21 12 0 13
22 13 13 0'''

Limb_Length(j, length_matrix)

2

In [4]:
'''
Code Challenge: Implement AdditivePhylogeny to solve the Distance-Based Phylogeny Problem.
Input: An integer n followed by a space-separated n x n distance matrix.
Output: A weighted adjacency list for the simple tree fitting this matrix.
'''

def Attached_Limb(length_matrix, j):
    for i in range(length_matrix.shape[0]):
        for k in range(length_matrix.shape[0]):
            if (i != j) & (k != j) :
                if length_matrix[i, k] == length_matrix[i, j - 1] + length_matrix[j - 1, k]:
                    return(i, k)

def Find_Path(nodes, current, final, path, visited, final_path):
    path = path + [current]
    visited.append(current)
    neighbor_nodes = nodes[current].keys()
    if current == final:
        final_path.extend(path) 
        return

    unvisited_neighbor_nodes = set(neighbor_nodes) - set(visited)
    if len(unvisited_neighbor_nodes) == 0:
        return
    
    for unvisited_neighbor_node in list(unvisited_neighbor_nodes):
        Find_Path(nodes,int(unvisited_neighbor_node),final,path,visited, final_path)

    return final_path
 
def add_to_graph(length_matrix,nodes,n,m,i,k,x):
    visited = []
    final_path = []
    
    i_k_path = Find_Path(nodes,i,k,[],visited,final_path)   
    total_length = 0

    for index in range(len(i_k_path) - 1):
        current_node   = i_k_path[index]
        next_node      = i_k_path[index + 1]
        length_between = nodes[current_node][next_node]
        total_length   = total_length + length_between

        if total_length == x:
            limb_length = Limb_Length(n, length_matrix)
            nodes[next_node][n] = limb_length
            nodes[n] = {next_node:limb_length}
            return nodes
        
        elif total_length > x:
            length1 = x - (total_length - length_between)
            length2 = total_length - x

            limb_length = Limb_Length(n , length_matrix)

            nodes[current_node].pop(next_node)
            nodes[next_node]   .pop(current_node)

            nodes[current_node][m[0]] = length1
            nodes[next_node][m[0]]    = length2
            nodes[m[0]] = {current_node:length1, next_node:length2}

            nodes[m[0]][n] = limb_length
            nodes[n]       = {m[0]:limb_length}
            m[0]           = m[0] + 1
            return nodes
    return nodes

def AdditivePhylogeny(length_matrix,n,m):
    '''if type(length_matrix) == str:
        length_matrix = length_matrix.replace('\n', ' ')
        length_matrix = length_matrix.split(' ')
        length_matrix = list(map(int, length_matrix))
        length_matrix = np.array(length_matrix).reshape(n, n)'''
    
    if n == 1:
        nodes = {}
        nodes[1] = {0:length_matrix[0, 1]}
        nodes[0] = {1:length_matrix[0, 1]}
        return nodes
    
    limb_length = Limb_Length(n , length_matrix)

    sub_matrix = copy.deepcopy(length_matrix)

    for j in range(n):

        sub_matrix[j, n] = sub_matrix[j, n] - limb_length
        sub_matrix[n, j] = sub_matrix[j, n]   

    (i,k) = Attached_Limb(sub_matrix, n)

    x = sub_matrix[i, n]

    sub_matrix = sub_matrix[: -1, : -1]
        
    nodes = AdditivePhylogeny(sub_matrix,n-1,m)

    nodes = add_to_graph(length_matrix, nodes, n, m, i, k, x)
    return nodes

#Test
n = 4
length_matrix = '''0 13 21 22
13 0 12 13
21 12 0 13
22 13 13 0'''
if type(length_matrix) == str:
    length_matrix = length_matrix.replace('\n', ' ')
    length_matrix = length_matrix.split(' ')
    length_matrix = list(map(int, length_matrix))
    length_matrix = np.array(length_matrix).reshape(n, n)

tmp = AdditivePhylogeny(length_matrix, n - 1,[n])    
for key1, values1 in tmp.items():
    for key2, value2 in values1.items():
        print(str(key1) + '->' + str(key2) + ':' + str(value2))

1->4:2
0->4:11
4->0:11
4->1:2
4->5:4
2->5:6
5->4:4
5->2:6
5->3:7
3->5:7


# Week 2: More Algorithms for Constructing Trees from Distance Matrices

In [5]:
'''
Code Challenge: Implement UPGMA.
Input: An integer n followed by a space separated n x n distance matrix.
Output: An adjacency list for the ultrametric tree returned by UPGMA. 
'''

def UPGMA(n, length_matrix):
    if type(length_matrix) == str:
        length_matrix = length_matrix.replace('\n', ' ')
        length_matrix = length_matrix.split(' ')
        while '' in length_matrix:
            length_matrix.remove('')
        length_matrix = list(map(int, length_matrix))
        length_matrix = np.array(length_matrix).reshape(n, n)
        
    
    clusters = list(range(n))
    ages = dict()
    graph = dict()
    n_node_cluster = dict()
    
    for node in clusters:
        ages[node] = 0
        n_node_cluster[node] = 1
        
    while len(clusters) != 1:
        
        n = length_matrix.shape[0]
        
        # find minimun length
        min_length   = float('Inf')
        for i in range(n):
            for j in range(n):
                if (length_matrix[i, j] != 0) & (length_matrix[i, j] < min_length) & (i < j):
                    min_length   = length_matrix[i, j]
                    min_location = [i, j]
                    
        i_index, j_index = min_location
        
        i = clusters[i_index]
        j = clusters[j_index]
        
        #make new cluster
        new_cluster = max(clusters) + 1
        
        #update new cluster's n_node
        n_node_cluster[new_cluster] = n_node_cluster[i] + n_node_cluster[j]
        
        #update cluster list
        clusters.remove(i)
        clusters.remove(j)
        clusters.append(new_cluster)
        
        #update ages
        ages[new_cluster] = length_matrix[i_index, j_index] / 2
        
        #add new cluster to graph
        if new_cluster in graph :
            graph[new_cluster][i] = ages[new_cluster] - ages[i]
            graph[new_cluster][j] = ages[new_cluster] - ages[j]
        else:
            graph[new_cluster] = {i : (ages[new_cluster] - ages[i]), j : (ages[new_cluster] - ages[j])}
        if i in graph :
            graph[i][new_cluster] = ages[new_cluster] - ages[i]
        else: 
            graph[i] = {new_cluster : (ages[new_cluster] - ages[i])}
        if j in graph :
            graph[j][new_cluster] = ages[new_cluster] - ages[j]
        else: 
            graph[j] = {new_cluster : (ages[new_cluster] - ages[j])}
        
        new_col = []
        for vi, vj in zip(length_matrix[:,i_index], length_matrix[:,j_index]):
            if (vi * vj) != 0:
                new_value = (vi * n_node_cluster[i] + vj * n_node_cluster[j]) / (n_node_cluster[i] + n_node_cluster[j])
                new_col.append(new_value)
        
        #update matrix
        length_matrix = np.delete(length_matrix, [i_index, j_index], 0)
        length_matrix = np.delete(length_matrix, [i_index, j_index], 1)
        
        length_matrix = np.vstack((length_matrix,np.array(new_col).reshape(1, len(new_col))))
        new_col.append(0)
        length_matrix = np.hstack((length_matrix,np.array(new_col).reshape(len(new_col), 1)))
            
    return(graph)

# Test
n = 4
length_matrix = '''0 20 17 11
20 0 20 13
17 20 0 10
11 13 10 0'''

tmp = UPGMA(n, length_matrix)

for key1, values1 in tmp.items():
    for key2, value2 in values1.items():
        print(str(key1) + '->' + str(key2) + ':' + str(value2))

4->2:5.0
4->3:5.0
4->5:2.0
2->4:5.0
3->4:5.0
5->0:7.0
5->4:2.0
5->6:1.83333333333
0->5:7.0
6->1:8.83333333333
6->5:1.83333333333
1->6:8.83333333333


In [6]:
'''
Code Challenge: Implement NeighborJoining.
Input: An integer n, followed by an n x n distance matrix.
Output: An adjacency list for the tree resulting from applying the neighbor-joining algorithm. 
'''

def NeighborJoining(n, length_matrix, nodes, m):

    graph = dict()
    limb_length = dict()
    if n == 2:
        graph[nodes[0]] = {nodes[1]: length_matrix[0, 1]}
        graph[nodes[1]] = {nodes[0]: length_matrix[1, 0]}
        return(graph)
    
    total_distance = {}
    for i in range(n):
        total_distance[i] = np.sum(length_matrix[i,:])
    
    # generate joining matrix 
    matrix_star = np.full([n,n], 0, float)
    for i in range(n):
        for j in range(n):
            if i != j:
                matrix_star[i, j] = (n - 2) * length_matrix[i, j] - total_distance[i] - total_distance[j]
    
    # find minimun value in joining matrix
    min_length   = float('Inf')
    for i in range(n):
        for j in range(n):
            if (matrix_star[i, j] != 0) & (matrix_star[i, j] < min_length) & (i < j):
                min_length   = matrix_star[i, j]
                min_location = [i, j]
    i_index, j_index = min_location
    
    i = nodes[i_index]
    j = nodes[j_index]
    
    # calculate delta for limb length
    delta = (total_distance[i_index] - total_distance[j_index]) / (n - 2)
    
    limb_length[i_index] = (length_matrix[i_index, j_index] + delta) / 2
    limb_length[j_index] = (length_matrix[i_index, j_index] - delta) / 2
    
    #make new node
    new_node = m 
    m = m + 1
    
    # update node list
    nodes.append(new_node)
    nodes.remove(i)
    nodes.remove(j)
    
    # update length matrix
    new_col = []
    for k in range(n):
        new_value = (length_matrix[k, i_index] + length_matrix[k, j_index] - length_matrix[i_index, j_index]) / 2
        new_col.append(new_value)
    length_matrix = np.vstack((length_matrix,np.array(new_col).reshape(1, len(new_col))))
    new_col.append(0)
    length_matrix = np.hstack((length_matrix,np.array(new_col).reshape(len(new_col), 1)))
    
    length_matrix = np.delete(length_matrix, [i_index, j_index], 0)
    length_matrix = np.delete(length_matrix, [i_index, j_index], 1)
    
    # iteration
    graph = NeighborJoining(n - 1, length_matrix, nodes, m)
    
    # update graph
    if new_node in graph :
        graph[new_node][i] = limb_length[i_index]
        graph[new_node][j] = limb_length[j_index]
    else:
        graph[new_node] = {i : limb_length[i_index], j : limb_length[j_index]}
    if i in graph :
        graph[i][new_node] = limb_length[i_index]
    else: 
        graph[i] = {new_node : limb_length[i_index]}
    if j in graph :
        graph[j][new_node] = limb_length[j_index]
    else: 
        graph[j] = {new_node : limb_length[j_index]}  
    
    return(graph)

# Test
n = 4
length_matrix = '''0 23 27 20
23 0 30 28
27 30 0 30
20 28 30 0'''

nodes = list(range(n))
m = n
if type(length_matrix) == str:
    length_matrix = length_matrix.replace('\n', ' ')
    length_matrix = length_matrix.split(' ')
    while '' in length_matrix:
        length_matrix.remove('')
    length_matrix = list(map(int, length_matrix))
    length_matrix = np.array(length_matrix).reshape(n, n)

    
tmp = NeighborJoining(n, length_matrix, nodes, m)
for key1, values1 in tmp.items():
    for key2, value2 in values1.items():
        print(str(key1) + '->' + str(key2) + ':' + str(value2))

4->5:2.0
4->0:8.0
4->3:12.0
5->4:2.0
5->1:13.5
5->2:16.5
1->5:13.5
2->5:16.5
0->4:8.0
3->4:12.0


# Week 3: Constructing Evolutionary Trees from Characters

In [7]:
'''
Code Challenge: Implement SmallParsimony to solve the Small Parsimony Problem.
Input: An integer n followed by an adjacency list for a rooted binary tree with n leaves labeled by DNA strings.
Output: The minimum parsimony score of this tree, followed by the adjacency list of a tree corresponding to labeling
     internal nodes by DNA strings in order to minimize the parsimony score of the tree. 
'''

def SmallParsimony(tree, n_leaf):
    tree_graph  = dict()
    tags   = dict()
    n_character = 0
    current_node = 0
    node_character = dict()
    
    
    if type(tree) == str:
        tree = tree.split('\n')
        for line in tree:
            line  = line.split('->')
            upper = int(line[0])
            
            # tag 1 for processed(have character), 0 for un-processed
            try:
                bottom       = int(line[1])
                tags[bottom] = 0
                
                if upper in tree_graph:
                    tree_graph[upper].append(bottom)
                else:
                    tree_graph[upper] = [bottom]
                    
            except:
                bottom       = line[1]
                node_character[current_node] = bottom
                tags[current_node] = 1
                
                if upper in tree_graph:
                    tree_graph[upper].append(current_node)
                else:
                    tree_graph[upper] = [current_node]
                    
                current_node = current_node + 1
                n_character  = len(bottom)
                
                
    # find root
    for node in tree_graph.keys():
        if node not in tags:
            root = node
            break
    tags[root] = 0
    
    # make a matrix to record scores, character_score[node, character_index(ACGT), position of character]. 
    character_score = np.full((root + 1, 4, n_character), float('Inf'), float)
    
    # initalize the character_score matrix
    for node, tag in tags.items():
        if tag == 1:
            characters = node_character[node]
            for index in range(len(characters)):
                character       = characters[index]
                character_index = 'ACGT'.index(character)
                character_score[node, character_index, index] = 0
    
    # fill the character_score matrix
    while 0 in tags.values():
        for node, tag in tags.items():
            if tag == 0:
                down_node_1, down_node_2 = tree_graph[node]
                if (tags[down_node_1] * tags[down_node_2]) == 1 :
                    for position_index in range(n_character):
                        for character_index in range(4):
                            min_score = float('Inf')
                            for character_index_1 in range(4):
                                for character_index_2 in range(4):
                                    if character_index_1 == character_index:
                                        delta_1 = 0
                                    else:
                                        delta_1 = 1

                                    if character_index_2 == character_index:
                                        delta_2 = 0
                                    else:
                                        delta_2 = 1

                                    score = character_score[down_node_1, character_index_1, position_index] + character_score[down_node_2, character_index_2, position_index] + delta_1 + delta_2
                                    if score < min_score:
                                        min_score = score
                            character_score[node, character_index, position_index] = min_score
                    tags[node] = 1
                        
    # find root characters                    
    root_character = ''
    for position_index in range(n_character):
        score_list      = list(character_score[root, :, position_index])
        character_index = score_list.index(min(score_list))
        character       = 'ACGT'[character_index]
        root_character  = root_character + character
    node_character[root] = root_character
    
    # trackback to find all other characters
    while len(node_character) != root + 1:
        for node, down_nodes in tree_graph.items():
            if (node in node_character):
                if (down_nodes[0] not in node_character) & (down_nodes[1] not in node_character):
                    characters_1 = ''
                    characters_2 = ''
                    
                    for position_index in range(n_character):
                        character       = node_character[node][position_index]
                        character_index = 'ACGT'.index(character)
                        socre           = character_score[node, character_index, position_index]
                        
                        for character_index_1 in range(4):
                            for character_index_2 in range(4):
                                if character_index_1 == character_index:
                                    delta_1 = 0
                                else:
                                    delta_1 = 1

                                if character_index_2 == character_index:
                                    delta_2 = 0
                                else:
                                    delta_2 = 1
                                
                                if socre == (character_score[down_nodes[0], character_index_1, position_index] + character_score[down_nodes[1], character_index_2, position_index] + delta_1 + delta_2) :
                                    min_index_1, min_index_2 = character_index_1, character_index_2
                                    
                        character_1  = 'ACGT'[min_index_1]
                        character_2  = 'ACGT'[min_index_2]
                        characters_1 = characters_1 + character_1
                        characters_2 = characters_2 + character_2
                        
                    node_character[down_nodes[0]] = characters_1
                    node_character[down_nodes[1]] = characters_2
                
                if (down_nodes[0] not in node_character) & (down_nodes[1] in node_character):
                    characters_1 = ''
                    
                    for position_index in range(n_character):
                        character       = node_character[node][position_index]
                        character_index = 'ACGT'.index(character)
                        socre           = character_score[node, character_index, position_index]
                        
                        character_2       = node_character[down_nodes[1]][position_index]
                        character_index_2 = 'ACGT'.index(character_2)
                        
                        for character_index_1 in range(4):
                            if character_index_1 == character_index:
                                delta_1 = 0
                            else:
                                delta_1 = 1

                            if character_index_2 == character_index:
                                delta_2 = 0
                            else:
                                delta_2 = 1
                            
                            if socre == (character_score[down_nodes[0], character_index_1, position_index] + character_score[down_nodes[1], character_index_2, position_index] + delta_1 + delta_2) :
                                min_index_1 = character_index_1
                        
                        character_1  = 'ACGT'[min_index_1]
                        characters_1 = characters_1 + character_1
                    node_character[down_nodes[0]] = characters_1
                
                if (down_nodes[0] in node_character) & (down_nodes[1] not in node_character):
                    characters_2 = ''
                    
                    for position_index in range(n_character):
                        character       = node_character[node][position_index]
                        character_index = 'ACGT'.index(character)
                        socre           = character_score[node, character_index, position_index]
                        
                        character_1       = node_character[down_nodes[0]][position_index]
                        character_index_1 = 'ACGT'.index(character_1)
                        
                        for character_index_2 in range(4):
                            if character_index_1 == character_index:
                                delta_1 = 0
                            else:
                                delta_1 = 1

                            if character_index_2 == character_index:
                                delta_2 = 0
                            else:
                                delta_2 = 1
                            
                            if socre == (character_score[down_nodes[0], character_index_1, position_index] + character_score[down_nodes[1], character_index_2, position_index] + delta_1 + delta_2) :
                                min_index_2 = character_index_2
                        
                        character_2  = 'ACGT'[min_index_2]
                        characters_2 = characters_2 + character_2
                    node_character[down_nodes[1]] = characters_2
    
    # connect node to each other
    new_graph = dict()
    for node, down_nodes in tree_graph.items():
        if down_nodes[0] not in tree_graph:
            new_graph[down_nodes[0]] = [node]
        else:
            if node not in tree_graph[down_nodes[0]]:
                tree_graph[down_nodes[0]].append(node)
                
        if down_nodes[1] not in tree_graph:
            new_graph[down_nodes[1]] = [node]
        else:
            if node not in tree_graph[down_nodes[1]]:
                tree_graph[down_nodes[1]].append(node)
    
    tree_graph = {**tree_graph, **new_graph}
    
    # calculate score for root
    final_score = 0
    for position_index in range(n_character):
        character       = node_character[root][position_index]
        character_index = 'ACGT'.index(character)
        position_score  = character_score[root, character_index, position_index]
        final_score     = final_score + position_score
    

    return(tree_graph, node_character, final_score)

def HammingDistance(string1, string2):
    mismatch = 0
    for base1, base2 in zip (string1, string2):
        if base1 != base2: 
            mismatch = mismatch + 1
    return (mismatch)


# Test
n_leaf = 4
tree   = '''4->CAAATCCC
4->ATTGCGAC
5->CTGCGCTG
5->ATGGACGA
6->4
6->5'''

tree_graph, node_character, final_score = SmallParsimony(tree, n_leaf)

print(int(final_score))
for node_index, down_node_indexs in tree_graph.items():
    node = node_character[node_index]
    for down_node_index in down_node_indexs:
        down_node = node_character[down_node_index]
        print(str(node) + '->' + str(down_node) + ':' + str(HammingDistance(node, down_node)))

16
ATAGTCAC->CAAATCCC:4
ATAGTCAC->ATTGCGAC:3
ATAGTCAC->ATAGACAA:2
ATGGACTA->CTGCGCTG:4
ATGGACTA->ATGGACGA:1
ATGGACTA->ATAGACAA:2
ATAGACAA->ATAGTCAC:2
ATAGACAA->ATGGACTA:2
CAAATCCC->ATAGTCAC:4
ATTGCGAC->ATAGTCAC:3
CTGCGCTG->ATGGACTA:4
ATGGACGA->ATGGACTA:1


In [8]:
'''
Code Challenge: Solve the Small Parsimony in an Unrooted Tree Problem.
Input: An integer n followed by an adjacency list for an unrooted binary tree with n leaves labeled by DNA strings.
Output: The minimum parsimony score of this tree, followed by the adjacency list of the tree corresponding to labeling
     internal nodes by DNA strings in order to minimize the parsimony score of the tree.
'''

def SmallParsimony_Unrooted(tree, n_leaf):
    tree_graph  = dict()
    tags   = dict()
    n_character = 0
    current_node = 0
    node_character = dict()
    
    
    if type(tree) == str:
        tree = tree.split('\n')
        for line in tree:
            line  = line.split('->')
            if line[0].isdigit() :
                upper = int(line[0])

                # tag 1 for processed(have character), 0 for un-processed
                try:
                    bottom       = int(line[1])
                    tags[bottom] = 0

                    if upper in tree_graph:
                        tree_graph[upper].append(bottom)
                    else:
                        tree_graph[upper] = [bottom]

                except:
                    bottom       = line[1]
                    node_character[current_node] = bottom
                    tags[current_node] = 1

                    if upper in tree_graph:
                        tree_graph[upper].append(current_node)
                    else:
                        tree_graph[upper] = [current_node]

                    current_node = current_node + 1
                    n_character  = len(bottom)
    
    # make a fake root
    root = max(tree_graph.keys()) + 1
    tree_graph[root] = [max(tree_graph.keys())]
    tree_graph[root].append(max(tree_graph[root - 1]))
    tree_graph[root - 1].remove(max(tree_graph[root - 1]))
    tags[root] = 0
    
    # make the tree have same pattern as rooted tree
    remove_check       = copy.deepcopy(tags)
    remove_check[root]     = 1

    while sum(remove_check.values()) != (len(remove_check.values()) - 1):
        for node, down_nodes in tree_graph.items():
            if len(down_nodes) == 3:
                if remove_check[node] == 0:
                    if (remove_check[down_nodes[0]] + remove_check[down_nodes[1]] + remove_check[down_nodes[2]]) == 2:
                        for down_node in down_nodes:
                            if remove_check[down_node] == 0:
                                tree_graph[node].remove(down_node)
                                break
                        remove_check[node]      = 1

    
    
    # make a matrix to record scores, character_score[node, character_index(ACGT), position of character]. 
    character_score = np.full((root + 1, 4, n_character), float('Inf'), float)
    
    # initalize the character_score matrix
    for node, tag in tags.items():
        if tag == 1:
            characters = node_character[node]
            for index in range(len(characters)):
                character       = characters[index]
                character_index = 'ACGT'.index(character)
                character_score[node, character_index, index] = 0
    
    #print(tree_graph, tags, character_score)
    # fill the character_score matrix
    while 0 in tags.values():
        for node, tag in tags.items():
            if tag == 0:
                down_node_1, down_node_2 = tree_graph[node]
                if (tags[down_node_1] * tags[down_node_2]) == 1 :
                    for position_index in range(n_character):
                        for character_index in range(4):
                            min_score = float('Inf')
                            for character_index_1 in range(4):
                                for character_index_2 in range(4):
                                    if character_index_1 == character_index:
                                        delta_1 = 0
                                    else:
                                        delta_1 = 1

                                    if character_index_2 == character_index:
                                        delta_2 = 0
                                    else:
                                        delta_2 = 1

                                    score = character_score[down_node_1, character_index_1, position_index] + character_score[down_node_2, character_index_2, position_index] + delta_1 + delta_2
                                    if score < min_score:
                                        min_score = score
                            character_score[node, character_index, position_index] = min_score
                    tags[node] = 1
                        
    # find root characters                    
    root_character = ''
    for position_index in range(n_character):
        score_list      = list(character_score[root, :, position_index])
        character_index = score_list.index(min(score_list))
        character       = 'ACGT'[character_index]
        root_character  = root_character + character
    node_character[root] = root_character
    
    # trackback to find all other characters
    while len(node_character) != root + 1:
        for node, down_nodes in tree_graph.items():
            if (node in node_character):
                if (down_nodes[0] not in node_character) & (down_nodes[1] not in node_character):
                    characters_1 = ''
                    characters_2 = ''
                    
                    for position_index in range(n_character):
                        character       = node_character[node][position_index]
                        character_index = 'ACGT'.index(character)
                        socre           = character_score[node, character_index, position_index]
                        
                        for character_index_1 in range(4):
                            for character_index_2 in range(4):
                                if character_index_1 == character_index:
                                    delta_1 = 0
                                else:
                                    delta_1 = 1

                                if character_index_2 == character_index:
                                    delta_2 = 0
                                else:
                                    delta_2 = 1
                                
                                if socre == (character_score[down_nodes[0], character_index_1, position_index] + character_score[down_nodes[1], character_index_2, position_index] + delta_1 + delta_2) :
                                    min_index_1, min_index_2 = character_index_1, character_index_2
                                    
                        character_1  = 'ACGT'[min_index_1]
                        character_2  = 'ACGT'[min_index_2]
                        characters_1 = characters_1 + character_1
                        characters_2 = characters_2 + character_2
                        
                    node_character[down_nodes[0]] = characters_1
                    node_character[down_nodes[1]] = characters_2
                
                if (down_nodes[0] not in node_character) & (down_nodes[1] in node_character):
                    characters_1 = ''
                    
                    for position_index in range(n_character):
                        character       = node_character[node][position_index]
                        character_index = 'ACGT'.index(character)
                        socre           = character_score[node, character_index, position_index]
                        
                        character_2       = node_character[down_nodes[1]][position_index]
                        character_index_2 = 'ACGT'.index(character_2)
                        
                        for character_index_1 in range(4):
                            if character_index_1 == character_index:
                                delta_1 = 0
                            else:
                                delta_1 = 1

                            if character_index_2 == character_index:
                                delta_2 = 0
                            else:
                                delta_2 = 1
                            
                            if socre == (character_score[down_nodes[0], character_index_1, position_index] + character_score[down_nodes[1], character_index_2, position_index] + delta_1 + delta_2) :
                                min_index_1 = character_index_1
                        
                        character_1  = 'ACGT'[min_index_1]
                        characters_1 = characters_1 + character_1
                    node_character[down_nodes[0]] = characters_1
                
                if (down_nodes[0] in node_character) & (down_nodes[1] not in node_character):
                    characters_2 = ''
                    
                    for position_index in range(n_character):
                        character       = node_character[node][position_index]
                        character_index = 'ACGT'.index(character)
                        socre           = character_score[node, character_index, position_index]
                        
                        character_1       = node_character[down_nodes[0]][position_index]
                        character_index_1 = 'ACGT'.index(character_1)
                        
                        for character_index_2 in range(4):
                            if character_index_1 == character_index:
                                delta_1 = 0
                            else:
                                delta_1 = 1

                            if character_index_2 == character_index:
                                delta_2 = 0
                            else:
                                delta_2 = 1
                            
                            if socre == (character_score[down_nodes[0], character_index_1, position_index] + character_score[down_nodes[1], character_index_2, position_index] + delta_1 + delta_2) :
                                min_index_2 = character_index_2
                        
                        character_2  = 'ACGT'[min_index_2]
                        characters_2 = characters_2 + character_2
                    node_character[down_nodes[1]] = characters_2
    
    # connect node to each other
    new_graph = dict()
    for node, down_nodes in tree_graph.items():
        if down_nodes[0] not in tree_graph:
            new_graph[down_nodes[0]] = [node]
        else:
            if node not in tree_graph[down_nodes[0]]:
                tree_graph[down_nodes[0]].append(node)
                
        if down_nodes[1] not in tree_graph:
            new_graph[down_nodes[1]] = [node]
        else:
            if node not in tree_graph[down_nodes[1]]:
                tree_graph[down_nodes[1]].append(node)
    
    tree_graph = {**tree_graph, **new_graph}
    
    # calculate score for root
    final_score = 0
    for position_index in range(n_character):
        character       = node_character[root][position_index]
        character_index = 'ACGT'.index(character)
        position_score  = character_score[root, character_index, position_index]
        final_score     = final_score + position_score
    
    # remove the fake root and re-connect the node
    node_1, node_2 = tree_graph[root]
    tree_graph[node_1].remove(root)
    tree_graph[node_1].append(node_2)
    tree_graph[node_2].remove(root)
    tree_graph[node_2].append(node_1)
    tree_graph.pop(root)
    
    return(tree_graph, node_character, final_score)

# Test
n_leaf = 4
tree = '''TCGGCCAA->4
4->TCGGCCAA
CCTGGCTG->4
4->CCTGGCTG
CACAGGAT->5
5->CACAGGAT
TGAGTACC->5
5->TGAGTACC
4->5
5->4'''

tree_graph, node_character, final_score = SmallParsimony_Unrooted(tree, n_leaf)

print(int(final_score))
for node_index, down_node_indexs in tree_graph.items():
    node = node_character[node_index]
    for down_node_index in down_node_indexs:
        down_node = node_character[down_node_index]
        print(str(node) + '->' + str(down_node) + ':' + str(HammingDistance(node, down_node)))

17
CCTGGCAA->TCGGCCAA:3
CCTGGCAA->CCTGGCTG:2
CCTGGCAA->CAAGGAAT:4
CAAGGAAT->CACAGGAT:3
CAAGGAAT->TGAGTACC:5
CAAGGAAT->CCTGGCAA:4
TCGGCCAA->CCTGGCAA:3
CCTGGCTG->CCTGGCAA:2
CACAGGAT->CAAGGAAT:3
TGAGTACC->CAAGGAAT:5


In [9]:
'''
Code Challenge: Solve the Nearest Neighbors of a Tree Problem.
Input: Two internal nodes a and b specifying an edge e, followed by an adjacency list of an unrooted binary tree.
Output: Two adjacency lists representing the nearest neighbors of the tree with respect to e. Separate the
     adjacency lists with a blank line.
'''
def Nearest_Neighbors_Tree(a, b, tree):
    
    a=str(a)
    b=str(b)
    
    graph_tree = dict()
    if type(tree) == str:
        tree = tree.split('\n')
    for line in tree:
        line = line.split('->')
        if line[0] in graph_tree:
            graph_tree[line[0]].append(line[1])
        else:
            graph_tree[line[0]] = [line[1]]

    graph_tree[a].remove(b)
    graph_tree[b].remove(a)

    w, x = graph_tree[a]
    y, z = graph_tree[b]
    
    graph_tree[a].append(b)
    graph_tree[b].append(a)
    
    new_tree = copy.deepcopy(graph_tree)
    new_tree[a].remove(x)
    new_tree[x].remove(a)
    new_tree[b].remove(y)
    new_tree[y].remove(b)
    
    new_tree[a].append(y)
    new_tree[y].append(a)
    new_tree[b].append(x)
    new_tree[x].append(b)
    
    neighbor_tree = copy.deepcopy(graph_tree)
    neighbor_tree[a].remove(x)
    neighbor_tree[x].remove(a)
    neighbor_tree[b].remove(z)
    neighbor_tree[z].remove(b)
    
    neighbor_tree[a].append(z)
    neighbor_tree[z].append(a)
    neighbor_tree[b].append(x)
    neighbor_tree[x].append(b)
    
    return(new_tree, neighbor_tree)       
    
# Test
a, b = 5, 4
tree = '''0->4
4->0
1->4
4->1
2->5
5->2
3->5
5->3
4->5
5->4'''

new_tree, neighbor_tree = Nearest_Neighbors_Tree(a, b, tree)

for from_, tos in new_tree.items():
    for to in tos:
        print(str(from_) + '->' + str(to))
print('')
for from_, tos in neighbor_tree.items():
    for to in tos:
        print(str(from_) + '->' + str(to))  

0->5
4->1
4->5
4->3
1->4
2->5
5->2
5->4
5->0
3->4

0->4
4->0
4->5
4->3
1->5
2->5
5->2
5->4
5->1
3->4


In [10]:
'''
Code Challenge: Implement the nearest neighbor interchange heuristic for the Large Parsimony Problem.
Input: An integer n, followed by an adjacency list for an unrooted binary tree whose n leaves are labeled by DNA strings and
     whose internal nodes are labeled by integers.
Output: The parsimony score and unrooted labeled tree obtained after every step of the nearest neighbor interchange heuristic.
     Each step should be separated by a blank line.
'''

def Graph_to_str(tree_graph, node_character, leafs):
    list_tree = []
    for node_index, down_node_indexs in tree_graph.items():
        if node_index in leafs:
            node = node_character[node_index]
        else: node = str(node_index)
        for down_node_index in down_node_indexs:
            if down_node_index in leafs:
                down_node = node_character[down_node_index]
            else:
                down_node = str(down_node_index)
            list_tree.append(node + '->' + down_node)
    return('\n'.join(list_tree))

def NearestNeighborInterchange(tree, n_leaf):
    score = float('Inf')
    tree, node_character, new_score = SmallParsimony_Unrooted(tree, n_leaf)
    leafs          = list(range(n_leaf))
    internal_nodes = list(range(n_leaf, max(tree.keys()) + 1))
    new_tree       = tree
    new_char       = node_character
    #Graph_to_str(tree, node_character, leafs)
    while new_score < score:

        score = new_score
        tree  = new_tree
        node_character = new_char
        for internal_node in internal_nodes:
            down_nodes = tree[internal_node]
            for down_node in down_nodes:
                if down_node in internal_nodes:

                    neighbor_1, neighbor_2 = Nearest_Neighbors_Tree(internal_node, down_node, Graph_to_str(tree, node_character, leafs))
                    neighbor_1_str = Graph_to_str(neighbor_1, node_character, leafs)
                    neighbor_2_str = Graph_to_str(neighbor_2, node_character, leafs)
                    tree1, node_character1, neighbor_score1 = SmallParsimony_Unrooted(neighbor_1_str, n_leaf)
                    tree2, node_character2, neighbor_score2 = SmallParsimony_Unrooted(neighbor_2_str, n_leaf)
                    
                    if neighbor_score1 < new_score:
                        new_score = neighbor_score1
                        new_tree  = tree1
                        new_char  = node_character1
                    if neighbor_score2 < new_score:
                        new_score = neighbor_score2
                        new_tree  = tree2
                        new_char  = node_character2
    
        print(int(new_score))
        for node_index, down_node_indexs in new_tree.items():
            node = node_character[node_index]
            for down_node_index in down_node_indexs:
                down_node = node_character[down_node_index]
                print(str(node) + '->' + str(down_node) + ':' + str(HammingDistance(node, down_node)))
        print('')

# Test
n_leaf = 5
tree = '''GCAGGGTA->5
TTTACGCG->5
CGACCTGA->6
GATTCCAC->6
5->TTTACGCG
5->GCAGGGTA
5->7
TCCGTAGT->7
7->5
7->6
7->TCCGTAGT
6->GATTCCAC
6->CGACCTGA
6->7'''

NearestNeighborInterchange(tree, n_leaf)

22
TCAGCGTA->TTTACGCG:5
TCAGCGTA->GCAGGGTA:2
TCAGCGTA->TCAGCAGA:2
TCAGCAGA->TCAGCGTA:2
TCAGCAGA->TCCGTAGT:3
TCAGCAGA->CAACCTGA:4
CAACCTGA->GATTCCAC:6
CAACCTGA->CGACCTGA:1
CAACCTGA->TCAGCAGA:4
TTTACGCG->TCAGCGTA:5
GCAGGGTA->TCAGCGTA:2
TCCGTAGT->TCAGCAGA:3
GATTCCAC->CAACCTGA:6
CGACCTGA->CAACCTGA:1

21
TCTGCGGT->TTTACGCG:4
TCTGCGGT->TCCGTAGT:3
TCTGCGGT->GAACCCGA:6
GCAGCGGA->GCAGGGTA:2
GCAGCGGA->GATTCCAC:6
GCAGCGGA->GAACCCGA:3
GAACCCGA->CGACCTGA:3
GAACCCGA->TCTGCGGT:6
GAACCCGA->GCAGCGGA:3
TTTACGCG->TCTGCGGT:4
TCCGTAGT->TCTGCGGT:3
GCAGGGTA->GCAGCGGA:2
GATTCCAC->GCAGCGGA:6
CGACCTGA->GAACCCGA:3

21
TCTGCGGT->TTTACGCG:4
TCTGCGGT->TCCGTAGT:3
TCTGCGGT->GCTGCGGT:1
GCAGCGGA->GCAGGGTA:2
GCAGCGGA->CGACCTGA:4
GCAGCGGA->GCTGCGGT:2
GCTGCGGT->GATTCCAC:5
GCTGCGGT->TCTGCGGT:1
GCTGCGGT->GCAGCGGA:2
TTTACGCG->TCTGCGGT:4
TCCGTAGT->TCTGCGGT:3
GCAGGGTA->GCAGCGGA:2
CGACCTGA->GCAGCGGA:4
GATTCCAC->GCTGCGGT:5



# Week 4

In [11]:
'''
CODE CHALLENGE: Construct the graph of a spectrum.
Given: A space-delimited list of integers Spectrum.
Return: Graph(Spectrum).
'''

mass_AA_table = open('data/integer_mass_table.txt').read()
mass_AA_table = mass_AA_table.split('\n')

mass_AA = dict()
for line in mass_AA_table:
    line = line.split(' ')
    mass_AA[int(line[1])] = line[0]

def Spectrum_2_Graph(spectrum, mass_AA):
    if type(spectrum) == str:
        spectrum = spectrum.split(' ')
        spectrum.append(0)
        spectrum = list(map(int, spectrum))
    
    graph = dict()
    for mass1 in spectrum:
        for mass2 in spectrum:
            delta = mass1 - mass2
            if delta in mass_AA:
                if mass2 in graph:
                    graph[mass2][mass1] = mass_AA[delta]
                else:  
                    graph[mass2] = {mass1 : mass_AA[delta]}
    return(graph)

spectrum = '57 71 154 185 301 332 415 429 486'

graph = Spectrum_2_Graph(spectrum, mass_AA)

for key1, values1 in graph.items():
    for key2, value2 in values1.items():    
        print(str(key1) + '->' + str(key2) + ':' + value2)

0->57:G
0->71:A
57->154:P
57->185:Q
71->185:N
154->301:F
185->332:F
301->415:N
301->429:Q
332->429:P
415->486:A
429->486:G


In [12]:
'''
CODE CHALLENGE: Solve the Decoding an Ideal Spectrum Problem.
Given: A space-delimited list of integers Spectrum.
Return: An amino acid string that explains Spectrum.
'''

table = open('data/integer_mass_table.txt').read()
table = table.split('\n')
AA_mass = {}
for line in table:
    AA, mass    = line.split(' ')
    AA_mass[AA] = int(mass)

def LinearSpectrum(peptide_string, AA_mass):
    prefix_weight = [0]
    for AA in peptide_string:
        prefix_weight.append(prefix_weight[-1] + AA_mass[AA])
        
    linear_spectrum = [0]
    for i in range(len(peptide_string)):
        for j in range(i + 1, len(peptide_string) + 1):
            linear_spectrum.append(prefix_weight[j] - prefix_weight[i])
    linear_spectrum.sort()
    return(linear_spectrum)

def DecodingIdealSpectrum(spectrum, mass_AA, AA_mass):
    if type(spectrum) == str:
        spectrum = spectrum.split(' ')
        spectrum = [0] + spectrum
        spectrum = list(map(int, spectrum))
    
    graph = Spectrum_2_Graph(spectrum, mass_AA)
    
    for start in graph.keys():
        peptide_string = ''
        Find_Graph_Path(graph, start, peptide_string, spectrum)

def Find_Graph_Path(graph, start, peptide_string, target_spectrum):
    
    current_spectrum = LinearSpectrum(peptide_string, AA_mass)
    if current_spectrum[-1] == target_spectrum[-1]:
        
        check = 1
        
        for item in target_spectrum:
            if item not in current_spectrum:
                check = check * 0
                break
        if check == 1:
            print(peptide_string)

    if start in graph:
        mass2_graph = graph[start]
        for mass2, AA in mass2_graph.items():
            Find_Graph_Path(graph, mass2, peptide_string + AA, target_spectrum)

# Test
spectrum = '57 71 154 185 301 332 415 429 486'

DecodingIdealSpectrum(spectrum, mass_AA, AA_mass)

GPFNA
ANFPG


In [13]:
'''
CODE CHALLENGE: Solve the Converting a Peptide into a Peptide Vector Problem.
Given: An amino acid string P.
Return: The peptide vector of P (in the form of space-separated integers).
'''

def Peptide_2_Vector(peptide_string, AA_mass):
    peptide_vector = [0] * LinearSpectrum(peptide_string, AA_mass)[-1]
    
    prefix_mass = 0
    for AA in peptide_string:
        
        prefix_mass = prefix_mass + AA_mass[AA]
        peptide_vector[prefix_mass - 1] = 1
        
    return(peptide_vector)

# Test
peptide_string = 'NADA'

peptide_vector = Peptide_2_Vector(peptide_string, AA_mass)
peptide_vector = list(map(str, peptide_vector))

print(' '.join(peptide_vector))

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1


In [14]:
'''
CODE CHALLENGE: Solve the Converting a Peptide Vector into a Peptide Problem.
Given: A space-delimited binary vector P.
Return: An amino acid string whose binary peptide vector matches P. For masses with more than one amino acid, any choice may be used.
'''

def Vector_2_Peptide(peptide_vector, mass_AA):
    if type(peptide_vector) == str:
        peptide_vector = peptide_vector.split(' ')
        
    peptide_vector = list(map(int, peptide_vector))
    peptide_vector = np.array(peptide_vector)
    
    positions = list(np.where(peptide_vector == 1)[0] + 1)
    positions = [0] + positions
    
    peptide_string = ''
    for i in range(1, len(positions)):
        mass = positions[i] - positions[i - 1]
        AA   = mass_AA[mass]
        
        peptide_string = peptide_string + AA
        
    return(peptide_string)

# Test
peptide_vector = '0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1'

Vector_2_Peptide(peptide_vector, mass_AA)

'NADA'

In [15]:
'''
CODE CHALLENGE: Solve the Peptide Sequencing Problem.
Given: A space-delimited spectral vector Spectrum'.
Return: An amino acid string with maximum score against Spectrum'. For masses with more than one amino acid, any choice may be used.
'''

def Spectral_Vector_2_Peptide(spectral_vector, mass_AA, AA_mass):
    if type(spectral_vector) == str:
        spectral_vector = spectral_vector.split(' ')
        spectral_vector = list(map(int, spectral_vector))
    spectral_vector = [0] + spectral_vector
    
    score_vector    = [-float('Inf')] * len(spectral_vector)
    score_vector[0] = 0
    
    # find max score
    for position in range(len(score_vector)):
        for mass in mass_AA.keys():
            if position >= mass:
                if score_vector[position - mass] + spectral_vector[position] > score_vector[position]:
                    score_vector[position] = score_vector[position - mass] + spectral_vector[position]
    
    peptide_string = ''
    
    position = len(spectral_vector) - 1
    
    # backtrack
    while position != 0:
        current_score = score_vector[position]
        delta_score   = spectral_vector[position]
        last_score    = current_score - delta_score
        last_position = np.where(np.array(score_vector) == last_score)[0]
        masss         = list(position - last_position)
        mass = 0
        for m in masss:
            if m in mass_AA:
                if m > mass:
                    mass = m
        AA            = mass_AA[mass]
        
        peptide_string = AA + peptide_string
        position       = position - mass
        
    return(peptide_string)

# Test
spectral_vector = '-19 -4 12 19 22 22 -8 29 21 9 -20 -2 -20 3 -20 9 15 7 -8 -4 27 29 -11 4 -5 10 -8 0 -19 -13 26 1 14 -18 -20 -2 -5 25 9 21 2 14 11 -11 -3 -6 13 -2 16 10 -12 27 11 -20 17 -20 4 11 -7 30 -4 22 17 -12 0 3 18 -13 9 8 16 -17 -9 12 11 30 18 -3 -1 30 -15 16 13 25 -10 29 9 -12 28 11 -4 -16 -6 16 2 -2 19 -12 -19 -1 13 -16 19 -19 -18 21 23 13 -2 22 -12 -8 -17 -11 30 5 23 -5 8 25 -9 23 -15 19 18 8 15 7 14 13 25 20 0 5 18 7 11 4 23 -19 -11 28 -14 6 14 21 -13 4 -8 10 8 -13 22 -13 17 -8 -7 -3 27 2 10 5 -4 27 16 -16 3 11 20 1 17 -20 -6 30 26 8 26 -9 -20 -6 10 4 -1 -7 -15 18 24 16 -14 -4 -13 -11 -17 15 11 -2 -8 5 22 4 -9 5 30 -9 5 9 -9 26 9 25 9 -12 30 16 5 -4 -19 -4 21 -13 27 -18 20 -11 -4 -4 -3 3 11 -19 29 -7 0 24 -1 5 -10 30 6 -9 -12 5 23 26 6 2 20 4 27 27 9 4 17 28 19 -8 -11 -9 -5 -2 -16 12 -6 -9 -14 29 -5 30 23 -8 13 12 -17 -14 12 29 -8 17 28 6 -15 25 -19 -20 3 -10 -3 -12 19 18 -2 -3 -18 17 13 4 11 27 5 12 22 3 -6 -15 4 22 24 -11 -5 25 -12 -12 -5 -18 -1 -2 25 18 -4 10 -8 -20 6 1 17 -4 24 -18 6 -6 11 -18 26 5 13 30 -9 -19 -17 -15 24 2 -13 -18 13 -13 -15 4 7 -5 2 -11 -8 -3 -14 2 30 -1 14 -1 19 -4 -16 29 10 13 -10 -1 -15 15 1 -12 24 22 -13 -19 10 2 -7 4 -2 18 -3 -14 22 -1 2 13 16 17 4 28 18 -13 10 -16 14 5 -15 6 18 6 6 18 0 21 -8 -2 2 18 -5 -20 -3 0 1 6 -5 8 22 14 9 18 7 5 9 23 -11 -13 -12 11 19 9 0 24 -1 24 23 -5 5 22 21 26 -8 27 8 22 -10 3 28 -7 -3 -16 17 29 17 26 -18 18 -19 -13 -14 -10 10 -15 -20 12 -16 23 14 -6 2 1 25 -4 4 -9 -8 14 8 -3 17 -19 -6 18 18 14 8 21 -17 -13 21 24 12 11 22 9 8 -14 -20 6 -2 -7 28 3 -12 25 -6 21 -19 21 -20 5 20 -20 15 25 2 21 27 -7 -19 26 -2 7 30 -14 30 -20 0 -12 -16 -17 -10 -7 27 -19 25 16 -12 6 -9 20 19 25 12 20 30 27 -10 11 -6 22 3 -10 -3 0 -4 -11 23 12 16 25 5 -13 -12 -17 9 -17 -12 -17 27 14 12 10 25 -16 10 -13 28 24 28 1 13 -13 7 -9 -2 -18 8 16 15 4 -14 4 22 -19 -1 17 25 15 -3 6 -14 -16 29 27 3 18 -19 -14 12 -10 19 9 -14 29 -12 7 5 -11 24 9 3 16 0 23 7 14 2 6 -8 3 -1 9 30 -12 12 29 30 6 27 1 28 27 -17 17 -16 25 17 7 15 2 0 8 14 22 12 12 -5 12 9 -7 16 -17 -1 -20 29 -14 16 -19 26 24 -16 4 3 -7 4 -8 16 29 -6 23 14 -14 5 -20 0 1 -4 -1 -7 24 14 -11 12 18 12 21 -10 -18 2 -6 5 -13 10 11 29 2 21 -9 -17 2 29 23 22 0 -4 12 -12 -3 0 -2 2 9 -19 -10 1 11 -16 -13 30 -20 -15 30 2 -1 26 18 29 6 -11 4 -14 -12 -17 -19 26 -19 -10 -7 26 23 -19 20 29 -17 25 23 16 6'

Spectral_Vector_2_Peptide(spectral_vector, mass_AA, AA_mass)

'CAGSAGGCGP'

# Week 5: Resolving the T. rex Peptides Mystery?